In [ ]:
# Recently I’ve been working on a pet project for realistic data augmentation of ECG-like and other grid-based “paper” images. 
# I decided to try the same approach here using my small library, MeshAugmentor (https://github.com/pashaalex/mesh_augmentor)
# This notebook demonstrates how to apply geometric augmentations and also how to map points from the original image 
# into the augmented one — which can be useful for keypoint tasks, segmentation borders, or synthetic dataset generation.
# If this tool helps you in your experiments — feel free to use it, customize it, or share feedback.

!wget -q https://github.com/pashaalex/mesh_augmentor/releases/download/0.1.0/mesh_augmentor-linux.zip
!unzip mesh_augmentor-linux.zip

In [ ]:
import cv2
import csv
import math
import numpy as np
import matplotlib.pyplot as plt

from mesh_augmentor import *

FILE_PATH = "/kaggle/input/physionet-ecg-image-digitization/train/1006427285/1006427285.csv"
OUTPUT_SIZE = 640
PIXELS_PER_MM = 4
PAPER_SPEED_MM_S = 25
SAMPLING_RATE = 1000.0  # Hz
DRAW_DOTTED=False
SIGNAL_COLOR=(120, 120, 120)
SIGNAL_THICKNESS=2

lead_names = ["I","II","III","aVR","aVL","aVF","V1","V2","V3","V4","V5","V6"]

def load_leads(path):
    result = {name: [] for name in lead_names}

    with open(path, newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            for col in result:
                val = row[col].strip()
                if val:
                    result[col].append(float(val))
    return result

def draw_signal(img, 
                leads, 
                signal_len,
                signal_name,                 
                x0, y0,
                xscale = PIXELS_PER_MM * PAPER_SPEED_MM_S / SAMPLING_RATE, 
                yscale = PIXELS_PER_MM * 10.0, 
                label_y_offset = 5 * PIXELS_PER_MM # 5mm offset to draw lead name
):
    signal = leads[signal_name][:int(signal_len * SAMPLING_RATE)]
    
    x_prev, y_prev = x0, y0

    for i in range(len(signal) - 1):
        x_curr = int(x0 + xscale * i)
        y_curr = int(y0 - yscale * signal[i + 1])
        cv2.line(img, (x_prev, y_prev), (x_curr, y_curr), color=SIGNAL_COLOR, thickness=SIGNAL_THICKNESS)
        x_prev, y_prev = x_curr, y_curr

    cv2.putText(img, 
                signal_name, 
                (x0, y0 + label_y_offset), 
                fontFace = cv2.FONT_HERSHEY_SIMPLEX, 
                fontScale=0.9, 
                color = SIGNAL_COLOR, 
                thickness = SIGNAL_THICKNESS, 
                lineType = cv2.LINE_AA)
    
    return x_curr

def draw_ecg(
        leads,
        img_width=PIXELS_PER_MM * 270,
        img_height=PIXELS_PER_MM * 210,
        grid_line_thickness=1,
        draw_dotted=False,
        background_color=(244, 244, 244),
        mm_line_color=(252, 203, 202)):
    img = np.full((img_height, img_width, 3), background_color, dtype=np.uint8)
    if draw_dotted == 1:
        for x in range(0, img_width, PIXELS_PER_MM):
            for y in range(0, img_height, PIXELS_PER_MM):
                cv2.rectangle(img, (x, y), (x + grid_line_thickness, y + grid_line_thickness), thickness=-1, color=mm_line_color)
    else:
        for x in range(0, img_width, PIXELS_PER_MM):             
            cv2.line(img, (x, 0), (x, img_height), color=mm_line_color, thickness=grid_line_thickness)
        for y in range(0, img_height, PIXELS_PER_MM):
            cv2.line(img, (0, y), (img_width, y), color=mm_line_color, thickness=grid_line_thickness)
    # Big lines (each 5mm)
    for x in range(0, img_width, PIXELS_PER_MM * 5):
        cv2.line(img, (x, 0), (x, img_height), color=mm_line_color, thickness=grid_line_thickness * 2)
    for y in range(0, img_height, PIXELS_PER_MM * 5):
        cv2.line(img, (0, y), (img_width, y), color=mm_line_color, thickness=grid_line_thickness * 2)

    x_next = draw_signal(img, leads, 2.5, 'I', 15 * PIXELS_PER_MM, 90 * PIXELS_PER_MM)
    x_next = draw_signal(img, leads, 2.5, 'aVR', x_next, 90 * PIXELS_PER_MM)
    x_next = draw_signal(img, leads, 2.5, 'V1', x_next, 90 * PIXELS_PER_MM)
    x_next = draw_signal(img, leads, 2.5, 'V4', x_next, 90 * PIXELS_PER_MM)

    x_next = draw_signal(img, leads, 2.5, 'II', 15 * PIXELS_PER_MM, 125 * PIXELS_PER_MM)
    x_next = draw_signal(img, leads, 2.5, 'aVL', x_next, 125 * PIXELS_PER_MM)
    x_next = draw_signal(img, leads, 2.5, 'V2', x_next, 125 * PIXELS_PER_MM)
    x_next = draw_signal(img, leads, 2.5, 'V5', x_next, 125 * PIXELS_PER_MM)

    x_next = draw_signal(img, leads, 2.5, 'III', 15 * PIXELS_PER_MM, 160 * PIXELS_PER_MM)
    x_next = draw_signal(img, leads, 2.5, 'aVF', x_next, 160 * PIXELS_PER_MM)
    x_next = draw_signal(img, leads, 2.5, 'V3', x_next, 160 * PIXELS_PER_MM)
    x_next = draw_signal(img, leads, 2.5, 'V6', x_next, 160 * PIXELS_PER_MM)

    draw_signal(img, leads, 10.0, 'II', 15 * PIXELS_PER_MM, 195 * PIXELS_PER_MM)

    return img

leads = load_leads(FILE_PATH)
ecg = draw_ecg(leads)

h, w, _ = ecg.shape
wcnt = w // 27
hcnt = h // 21

# Configure optics/lighting/distortion/etc.
optics = Optics(F=35.0, L=66.7, R=8.0)
distortion = Distortion(use=True, k1=-0.5)
lighting = Lighting(
    use=True,
    x=40.0,
    y=-40.0,
    z=10.0,
    intensity=1.5,
    diameter=170.0,
    light_mix_koef=0.9 
)

bg_shadow = BackgroundShadow(use=True, bg_z=150, bottom_shadow_koef=0.5)
pose = CameraPose(tilt_x_rad=math.radians(2.0))

# Create mesh
mesh = MeshAugmentor(
    input_width=w, input_height=h,
    grid_w=wcnt, grid_h=hcnt,
    optics=optics, 
    distortion=distortion, 
    lighting=lighting,
    bg_shadow=bg_shadow, 
    pose=pose
)

apply_crumple_with_creases(
    mesh,
    K=20,
    angle_deg_range=(8.0, 24.0),
    band_px=90,
    falloff_sigma=18.0,
    z_jitter_std=0.05,
    z_scale=0.4
)

# cylinder
stride = wcnt + 1
for index, point in enumerate(mesh.points):
    xc = index % stride
    yc = index // stride
    point.z -= 20 * math.sin(3.14 * xc / wcnt)

mesh.rotate_z(math.radians(5))
mesh.rotate_x(math.radians(2))
mesh.fit_best_geometric(OUTPUT_SIZE, OUTPUT_SIZE, 0.9)

# Render (RGB + optional alpha/mask/uv)
outs = mesh.render(
    input_image=ecg,
    out_size=(OUTPUT_SIZE, OUTPUT_SIZE),
    background = np.full((224, 224, 3), (255, 255, 255), dtype=np.uint8),
    attachments=("rgb", "mask", "uv"),
)

# Reproject start point from source image coordinates 
# into augmented image coordinates
x_point, y_point = mesh.reproject_point(15 * PIXELS_PER_MM, 
                                  90 * PIXELS_PER_MM,
                                  OUTPUT_SIZE,
                                  OUTPUT_SIZE)

cv2.circle(outs.rgb, (int(x_point), int(y_point)), 8, (255, 0, 0), thickness=-1)

plt.imshow(outs.rgb)
plt.tight_layout()
plt.show()